
# Chapter 5



# Lab: Cross-Validation and the Bootstrap
In this lab, we explore the resampling techniques covered in this
chapter. Some of the commands in this lab may take a while to run on
your computer.

We again begin by placing most of our imports at this top level.

In [1]:
import numpy as np
import statsmodels.api as sm
from ISLP import load_data
from ISLP.models import (ModelSpec as MS,
                         summarize,
                         poly)
from sklearn.model_selection import train_test_split


There are several new imports needed for this lab.

In [2]:
from functools import partial # This allows us to freeze some arguments of a function in order to later call the function with only the remaining arguments, simplifying its calling.
from sklearn.model_selection import \
     (cross_validate,
      KFold,
      ShuffleSplit)
from sklearn.base import clone # To clone an estimator without it having been trained, but with the same parameters as the original.
from ISLP.models import sklearn_sm


## The Validation Set Approach
We explore the use of the validation set approach in order to estimate
the test error rates that result from fitting various linear models on
the  `Auto`  data set.

We use the function `train_test_split()` to split
the data into training and validation sets. As there are 392 observations,
we split into two equal sets of size 196 using the
argument `test_size=196`. It is generally a good idea to set a random seed
when performing operations like this that contain an
element of randomness, so that the results obtained can be reproduced
precisely at a later time. We set the random seed of the splitter
with the argument `random_state=0`. 

In [3]:
Auto = load_data('Auto')
Auto_train, Auto_valid = train_test_split(Auto,
                                         test_size=196,
                                         random_state=0)

Now we can fit a linear regression using only the observations corresponding to the training set `Auto_train`.

In [4]:
# MS, ModelSpec, is a transform.
# For all of this we will use the train set.

hp_mm = MS(['horsepower']) # We transform 1 column
X_train = hp_mm.fit_transform(Auto_train) # We transform (with also fit) the dataframe
y_train = Auto_train['mpg'] # Output! Should this not be also transformed?

# I wonder: On X_train, won't we also have the output column? Shouldn't we drop it?

# To the model, we will input the transformed training X.
model = sm.OLS(y_train, X_train) # Model! It requires input and output (training samples)
results = model.fit() # then we fit the model


We now use the `predict()` method of `results` evaluated on the model matrix for this model
created using the validation data set. We also calculate the validation MSE of our model.

In [5]:
X_valid = hp_mm.transform(Auto_valid) # We must perform the same transformation.
y_valid = Auto_valid['mpg']

valid_pred = results.predict(X_valid) # As we have the X, we want the predicted ys.

# And a simple metric would be the MSE for the validation set samples.
np.mean((y_valid - valid_pred)**2)


23.61661706966988

Hence our estimate for the validation MSE of  the linear regression
fit is $23.62$.

We can also estimate the validation error for
higher-degree polynomial regressions. We first provide a function `evalMSE()` that takes a model string as well
as a training and test set and returns the MSE on the test set.

In [6]:
def evalMSE(terms, # df?
            response, # name of the response column
            train, # train samples
            test): # test samples

   # define the transform for the terms
   mm = MS(terms)
   X_train = mm.fit_transform(train) # use it on the train samples.
   y_train = train[response] # from the train samples, take the output.

   # ms, the transform, has to also transform the test samples
   X_test = mm.transform(test)
   y_test = test[response]

   # The model is OLS, simple linear regression. We fit it using the train samples.
   results = sm.OLS(y_train, X_train).fit()
   # And we calculate the predictions
   test_pred = results.predict(X_test)
    
   # And return the MSE
   return np.mean((y_test - test_pred)**2)


Let’s use this function to estimate the validation MSE
using linear, quadratic and cubic fits. We use the `enumerate()`  function
here, which gives both the values and indices of objects as one iterates
over a for loop.

In [7]:
MSE = np.zeros(3) # We will store the MSEs here.

# We will use polynomials of 1, 2 and 3 degrees.
for idx, degree in enumerate(range(1, 4)):
    MSE[idx] = evalMSE([poly('horsepower', degree)], # the terms parameter
                       'mpg', #the output column
                       Auto_train, #the train set
                       Auto_valid) # the validation set.
MSE

#poly construcs polynomials of the features. For a given column, in this case 'horsepower',
# we get the polynomials we would use for the linear regression.


array([23.61661707, 18.76303135, 18.79694163])

These error rates are $23.62, 18.76$, and $18.80$, respectively. If we
choose a different training/validation split instead, then we
can expect somewhat different errors on the validation set.

In [8]:
Auto_train, Auto_valid = train_test_split(Auto,
                                          test_size=196,
                                          random_state=3)
MSE = np.zeros(3)
for idx, degree in enumerate(range(1, 4)):
    MSE[idx] = evalMSE([poly('horsepower', degree)],
                       'mpg',
                       Auto_train,
                       Auto_valid)
MSE

array([20.75540796, 16.94510676, 16.97437833])

Using this split of the observations into a training set and a validation set,
we find that the validation set error rates for the models with linear, quadratic, and cubic terms are $20.76$, $16.95$, and $16.97$, respectively.

These results are consistent with our previous findings: a model that
predicts `mpg` using a quadratic function of `horsepower`
performs better than a model that involves only a linear function of
`horsepower`, and there is no evidence of an improvement in using a cubic function of `horsepower`.

## Cross-Validation
In theory, the cross-validation estimate can be computed for any generalized
linear model.  {}
In practice, however, the simplest way to cross-validate in
Python is to use `sklearn`, which has a different interface or API
than `statsmodels`, the code we have been using to fit GLMs.

This is a problem which often confronts data scientists: "I have a function to do task $A$, and need to feed it into something that performs task $B$, so that I can compute $B(A(D))$, where $D$ is my data." When $A$ and $B$ don’t naturally speak to each other, this
requires the use of a *wrapper*.
In the `ISLP` package,
we provide 
a wrapper, `sklearn_sm()`, that enables us to easily use the cross-validation tools of `sklearn` with
models fit by `statsmodels`.

The class `sklearn_sm()` 
has  as its first argument
a model from `statsmodels`. It can take two additional
optional arguments: `model_str` which can be
used to specify a formula, and `model_args` which should
be a dictionary of additional arguments used when fitting
the model. For example, to fit a logistic regression model
we have to specify a `family` argument. This
is passed as `model_args={'family':sm.families.Binomial()}`.

Here is our wrapper in action:

In [9]:
# sklearn_sm is a wrapper
# It takes a statsmodel model, a str for a formula and arguments
# for the model as a dictionary
# It helps us prepare the statsmodel model to be used with
# scikitlearn's cross-validate

In [11]:
hp_model = sklearn_sm(sm.OLS, # we use the OLS model from statsmodels
                      MS(['horsepower'])) # and select a column?

X, Y = Auto.drop(columns=['mpg']), Auto['mpg'] # cleverly creates X and Y

# We call for cross-validate.
# this will calculate CV(k) after dividing the set into k groups/folds.
# how many ks?
# It also allows us to calculate specific metrics, unlike cross_val_score.
cv_results = cross_validate(hp_model, # it takes the model
                            X,
                            Y,
                            cv=Auto.shape[0]) # and the amount of CVs=k
cv_err = np.mean(cv_results['test_score'])
cv_err


24.23151351792922

The arguments to `cross_validate()` are as follows: an
object with the appropriate `fit()`, `predict()`,
and `score()` methods,  an
array of features `X` and a response `Y`. 
We also included an additional argument `cv` to `cross_validate()`; specifying an integer
$K$ results in $K$-fold cross-validation. We have provided a value 
corresponding to the total number of observations, which results in
leave-one-out cross-validation (LOOCV). The `cross_validate()`  function produces a dictionary with several components;
we simply want the cross-validated test score here (MSE), which is estimated to be 24.23.

In [19]:
cv_results

{'fit_time': array([0.00461173, 0.00305367, 0.00364566, 0.00312543, 0.0028913 ,
        0.00304651, 0.0028708 , 0.00298452, 0.00348902, 0.00296402,
        0.00322962, 0.00341821, 0.00300837, 0.003052  , 0.00309157,
        0.00286579, 0.00295353, 0.00281978, 0.00283766, 0.00279903,
        0.00325084, 0.00286031, 0.00309205, 0.00275683, 0.00335741,
        0.00315714, 0.0028069 , 0.0030818 , 0.0031414 , 0.00297403,
        0.00280643, 0.0029552 , 0.00325632, 0.00412917, 0.00421   ,
        0.00571012, 0.00363302, 0.00291371, 0.00350785, 0.00301099,
        0.0028193 , 0.00302601, 0.00321293, 0.00284529, 0.00286031,
        0.00277209, 0.00338674, 0.00307369, 0.00287628, 0.00274897,
        0.00334001, 0.00311089, 0.00290537, 0.00276566, 0.00319481,
        0.00304198, 0.00286889, 0.00275183, 0.00327873, 0.00296044,
        0.00284004, 0.00274229, 0.00301147, 0.00302815, 0.00280571,
        0.00278091, 0.00298691, 0.00304127, 0.00282001, 0.00279808,
        0.00274777, 0.00328183, 0.00

We can repeat this procedure for increasingly complex polynomial fits.
To automate the process, we again
use a for loop which iteratively fits polynomial
regressions of degree 1 to 5, computes the
associated cross-validation error, and stores it in the $i$th element
of the vector `cv_error`. The variable `d` in the for loop
corresponds to the degree of the polynomial. We begin by initializing the
vector. This command may take a couple of seconds to run.

In [21]:
cv_error = np.zeros(5)
H = np.array(Auto['horsepower'])
M = sklearn_sm(sm.OLS)

# Fit using polynomials of degree 1 to 5
for i, d in enumerate(range(1,6)):
    # Outer en este caso aplica la función power a cada elemento
    # de H apareado con cada elemento del arange.
    # Entonces a cada elemento de H le va a aplicar power de 
    # primero 1
    # luego 1 y 2
    # luego 1, 2 y 3..
    X = np.power.outer(H, np.arange(d+1))
    # Y para cada caso, hace un cross-validation LOOCV.
    M_CV = cross_validate(M, # toma el modelo
                          X, # el input para esta iteración, que va a tener Horsepower, Horsepower^2, etc
                          Y,
                          cv=Auto.shape[0])
    cv_error[i] = np.mean(M_CV['test_score'])
cv_error


array([24.23151352, 19.24821312, 19.33498406, 19.42443029, 19.03320648])

As in Figure 5.4, we see a sharp drop in the estimated test MSE between the linear and
quadratic fits, but then no clear improvement from using higher-degree polynomials.

Above we introduced the `outer()`  method of the `np.power()`
function.  The `outer()` method is applied to an operation
that has two arguments, such as `add()`, `min()`, or
`power()`.
It has two arrays as
arguments, and then forms a larger
array where the operation is applied to each pair of elements of the
two arrays. 

In [22]:
A = np.array([3, 5, 9])
B = np.array([2, 4])
np.add.outer(A, B)


array([[ 5,  7],
       [ 7,  9],
       [11, 13]])

In the CV example above, we used $K=n$, but of course we can also use $K<n$. The code is very similar
to the above (and is significantly faster). Here we use `KFold()` to partition the data into $K=10$ random groups. We use `random_state` to set a random seed and initialize a vector `cv_error` in which we will store the CV errors corresponding to the
polynomial fits of degrees one to five.

In [24]:
cv_error = np.zeros(5)
cv = KFold(n_splits=10,
           shuffle=True,
           random_state=0) # use same splits for each degree
for i, d in enumerate(range(1,6)):
    X = np.power.outer(H, np.arange(d+1))
    M_CV = cross_validate(M,
                          X,
                          Y,
                          cv=cv)
    cv_error[i] = np.mean(M_CV['test_score'])
cv_error


array([24.20766449, 19.18533142, 19.27626666, 19.47848403, 19.13720065])

Notice that the computation time is much shorter than that of LOOCV.
(In principle, the computation time for LOOCV for a least squares
linear model should be faster than for $K$-fold CV, due to the
availability of the formula (5.2)  for LOOCV;
however, the generic `cross_validate()`  function does not make
use of this formula.)  We still see little evidence that using cubic
or higher-degree polynomial terms leads to a lower test error than simply
using a quadratic fit.

The `cross_validate()` function is flexible and can take
different splitting mechanisms as an argument. For instance, one can use the `ShuffleSplit()` funtion to implement
the validation set approach just as easily as K-fold cross-validation.

In [26]:
validation = ShuffleSplit(n_splits=1,
                          test_size=196,
                          random_state=0)
results = cross_validate(hp_model,
                         Auto.drop(['mpg'], axis=1),
                         Auto['mpg'],
                         cv=validation);
results['test_score']


array([23.61661707])

One can estimate the variability in the test error by running the following:

In [28]:
validation = ShuffleSplit(n_splits=10,
                          test_size=196,
                          random_state=0)
results = cross_validate(hp_model,
                         Auto.drop(['mpg'], axis=1),
                         Auto['mpg'],
                         cv=validation)
results['test_score'].mean(), results['test_score'].std()


(23.802232661034168, 1.4218450941091842)

Note that this standard deviation is not a valid estimate of the
sampling variability of the mean test score or the individual scores, since the randomly-selected training
samples overlap and hence introduce correlations. But it does give an
idea of the Monte Carlo variation
incurred by picking different random folds.

## The Bootstrap
We illustrate the use of the bootstrap in the simple example
 {of Section 5.2,}  as well as on an example involving
estimating the accuracy of the linear regression model on the  `Auto`
data set.
### Estimating the Accuracy of a Statistic of Interest
One of the great advantages of the bootstrap approach is that it can
be applied in almost all situations. No complicated mathematical
calculations are required. While there are several implementations
of the bootstrap in Python, its use for estimating
standard error is simple enough that we write our own function
below for the case when our data is stored
in a dataframe.

To illustrate the bootstrap, we
start with a simple example.
The  `Portfolio`  data set in the `ISLP` package is described
in Section 5.2. _The goal is to estimate the
sampling variance of the parameter $\alpha$_ given in formula (5.7).  We will
create a function
`alpha_func()`, which takes as input a dataframe `D` assumed
to have columns `X` and `Y`, as well as a
vector `idx` indicating which observations should be used to
estimate 
$\alpha$. The function then outputs the estimate for $\alpha$ based on
the selected observations.

In [29]:
Portfolio = load_data('Portfolio')
def alpha_func(D, idx):
   """
   Assumes the dataframe D has columns X and Y.
   idx is an array of positions to consider
   Returns the sampling variance of a parameter alpha.
   """
   # Calculates covariance of X and Y, considering only
   # the correct indexes.
   cov_ = np.cov(D[['X','Y']].loc[idx], rowvar=False)
   
   return ((cov_[1,1] - cov_[0,1]) /
           (cov_[0,0]+cov_[1,1]-2*cov_[0,1]))


This function returns an estimate for $\alpha$
based on applying the minimum
    variance formula (5.7) to the observations indexed by
the argument `idx`.  For instance, the following command
estimates $\alpha$ using all 100 observations.

In [31]:
Portfolio.head()

,X,Y
0,-0.895251,-0.234924
1,-1.562454,-0.885176
2,-0.417090,0.271888
3,1.044356,-0.734198
4,-0.315568,0.841983


In [33]:
alpha_func(Portfolio, range(100)) # calculates alpha's sampling variance using only the first 100 samples

0.57583207459283

Next we randomly **select
100 observations from `range(100)`, with replacement**. This is equivalent
to constructing a new bootstrap data set and recomputing $\hat{\alpha}$
based on the new data set.

In [34]:
rng = np.random.default_rng(0)
alpha_func(Portfolio,
           rng.choice(100, # np.arange(100)
                      100, # take 100 samples from the population
                      replace=True))

0.6074452469619004

This process can be generalized to create a simple function `boot_SE()` for
computing the bootstrap standard error for arbitrary
functions that take only a data frame as an argument.

In [36]:
def boot_SE(func, # arbitrary function
            D, # array with X and Y columns
            n=None,
            B=1000, # bootstrap replications
            seed=0):
    """
    Calculates the bootstrap standard error for the function
    """
    rng = np.random.default_rng(seed)
    first_, second_ = 0, 0
    n = n or D.shape[0]
    
    for _ in range(B): # repeat B times (the amount of replications of the bootstrap)
        idx = rng.choice(D.index, # this is an array
                         n, # we pick n values (default: array size)
                         replace=True)
        value = func(D, idx) # using the selected indexes, we apply the function
        
        first_ += value # we add the function result
        second_ += value**2 # and the function result squared

    # After all replications are done, we calculate the standard error of the estimated function result
    return np.sqrt(second_ / B - (first_ / B)**2)

So:
1. We first calculated the value of a function using a Bootstrapping technique
    1. Take a number of observations with replacement from the original population.
    2. Use those samples to calculate the function itself, $\hat{\alpha}$.
2. We then calculated the standard error associated with that function
    1. Repeat B times, with B the amount of replications to use.
       1. Take n values from the population
       2. Calculate the function for those values, $\hat{\alpha}^{*i}$.
    2. Calculate the standard error for the value by the following equation

$$
SE_B(\hat{\alpha}) = \sqrt{\frac{1}{B-1} \sum_{r=1}^B \left(\hat{\alpha}^{*r} - \frac{1}{B} \sum_{r'=1}^B \hat{\alpha}^{*r'} \right)}
$$

Notice the use of `_` as a loop variable in `for _ in range(B)`. This is often used if the value of the counter is
unimportant and simply makes sure  the loop is executed `B` times.

Let’s use our function to evaluate the accuracy of our
estimate of $\alpha$ using $B=1{,}000$ bootstrap replications. 

In [43]:
alpha_SE = boot_SE(alpha_func,
                   Portfolio,
                   B=1000,
                   seed=0)
alpha_SE


0.09118176521277699

The final output shows that the bootstrap estimate for ${\rm SE}(\hat{\alpha})$ is $0.0912$.

### Estimating the Accuracy of a Linear Regression Model
The bootstrap approach **can be used to assess the variability of the
coefficient estimates and predictions from a statistical learning
method**. Here we use the bootstrap approach in order to assess the
variability of the estimates for $\beta_0$ and $\beta_1$, the
intercept and slope terms for the linear regression model that uses
`horsepower` to predict `mpg` in the  `Auto`  data set. We
will compare the estimates obtained using the bootstrap to those
obtained using the formulas for ${\rm SE}(\hat{\beta}_0)$ and
${\rm SE}(\hat{\beta}_1)$ described in Section 3.1.2.

To use our `boot_SE()` function, we must write a function (its
first argument)
that takes a data frame `D` and indices `idx`
as its only arguments. But here we want to bootstrap a specific
regression model, specified by a model formula and data. We show how
to do this in a few simple steps.

We start by writing a generic
function `boot_OLS()` for bootstrapping a regression model that takes a formula to
define the corresponding regression. We use the `clone()` function to
make a copy of the formula that can be refit to the new dataframe. This means
that any derived features such as those defined by `poly()`
(which we will see shortly),
will be re-fit on the resampled data frame.

In [52]:
def boot_OLS(model_matrix, response, D, idx):
    D_ = D.iloc[idx]
    Y_ = D_[response]
    X_ = clone(model_matrix).fit_transform(D_)
    return sm.OLS(Y_, X_).fit().params #returns the parameters!

This is not quite what is needed as the first argument to
`boot_SE()`. The first two arguments which specify the model will not change in the
bootstrap process, and we would like to *freeze* them.   The
function `partial()` from the `functools` module  does precisely this: it takes a function
as an argument, and freezes some of its arguments, starting from the
left. We use it to freeze the first two model-formula arguments of `boot_OLS()`.

In [53]:
hp_func = partial(boot_OLS, # The function
                  MS(['horsepower']), # The model matrix, in this case for a single column
                  'mpg') # the response column name


Typing `hp_func?` will show that it has two arguments `D`
and `idx` --- it is a version of `boot_OLS()` with the first
two arguments frozen --- and hence is ideal as the first argument for `boot_SE()`.

The `hp_func()` function can now be used in order to create
bootstrap estimates for the intercept and slope terms by randomly
sampling from among the observations with replacement. We first
demonstrate its utility on 10 bootstrap samples.

In [71]:
Auto = Auto.reset_index()

In [73]:
rng = np.random.default_rng(0)
# call 10 times the frozen function and make an 
# array with their results, which are the parameters
np.array([hp_func(# We have to give the boot_OLS function the D and the idx args
                  Auto, # D arg: the dataframe
                  rng.choice(392, # idx's we use, bootstrap style (with replacement)
                             392,
                             replace=True)) for _ in range(10)])

# So we decide to repeat 10 times the calling of boot_OLS.
# This boot_OLS function results on the parameters obtained from a fix using OLS.
# therefore, we have b0, b1 (intercept and coeff) after a linear regression.

# As we are performing a bootstrap, the linear regression is not performed on ALL
# possible samples, but on a subset of those samples.
# How many samples? n samples taken **with replacement**. So instead of all possible samples
# being used, just a bunch of them are.

# This approach is taken B times, with B values being calculated.
# In this case, 10.

# We then get an array of 10 arrays
# each array with two values: one for b0 and one for b1

array([[39.88064456, -0.1567849 ],
       [38.73298691, -0.14699495],
       [38.31734657, -0.14442683],
       [39.91446826, -0.15782234],
       [39.43349349, -0.15072702],
       [40.36629857, -0.15912217],
       [39.62334517, -0.15449117],
       [39.0580588 , -0.14952908],
       [38.66688437, -0.14521037],
       [39.64280792, -0.15555698]])

Next, we **use the `boot_SE()` {}  function to compute the standard
errors of 1,000 bootstrap estimates** for the intercept and slope terms.

In [74]:
hp_se = boot_SE(hp_func, # This is the boot_OLS with partial
                Auto, # This is again the dataframe
                B=1000, # we want the bootstrap value to be 1000
                seed=10)

# This will call boot_SE, which calls the function for
# performing boot_OLS not 10 times but 1000, and calculates the SE.
hp_se


intercept     0.848807
horsepower    0.007352
dtype: float64

This indicates that the bootstrap estimate for ${\rm SE}(\hat{\beta}_0)$ is
0.85, and that the bootstrap
estimate for ${\rm SE}(\hat{\beta}_1)$ is
0.0074.  As discussed in
Section 3.1.2, standard formulas can be used to compute
the standard errors for the regression coefficients in a linear
model. These can be obtained using the `summarize()`  function
from `ISLP.sm`.

In [75]:
hp_model.fit(Auto, Auto['mpg'])
model_se = summarize(hp_model.results_)['std err']
model_se


intercept     0.717
horsepower    0.006
Name: std err, dtype: float64

The standard error estimates for $\hat{\beta}_0$ and $\hat{\beta}_1$
obtained using the formulas  from Section 3.1.2  are
0.717 for the
intercept and
0.006 for the
slope. Interestingly, these are somewhat different from the estimates
obtained using the bootstrap.  Does this indicate a problem with the
bootstrap? In fact, it suggests the opposite.  Recall that the
standard formulas given in
 {Equation 3.8 on page 82}
rely on certain assumptions. For example,
they depend on the unknown parameter $\sigma^2$, the noise
variance. We then estimate $\sigma^2$ using the RSS. Now although the
formula for the standard errors do not rely on the linear model being
correct, the estimate for $\sigma^2$ does.  We see
 {in Figure 3.8 on page 108}  that there is
a non-linear relationship in the data, and so the residuals from a
linear fit will be inflated, and so will $\hat{\sigma}^2$.  Secondly,
the standard formulas assume (somewhat unrealistically) that the $x_i$
are fixed, and all the variability comes from the variation in the
errors $\epsilon_i$.  The **bootstrap approach does not rely on any of
these assumptions, and so it is likely giving a more accurate estimate
of the standard errors** of $\hat{\beta}_0$ and $\hat{\beta}_1$ than
the results from `sm.OLS`.

Below we compute the bootstrap standard error estimates and the
standard linear regression estimates that result from fitting the
quadratic model to the data. Since this model provides a good fit to
the data (Figure 3.8), there is now a better
correspondence between the bootstrap estimates and the standard
estimates of ${\rm SE}(\hat{\beta}_0)$, ${\rm SE}(\hat{\beta}_1)$ and
${\rm SE}(\hat{\beta}_2)$.

In [76]:
quad_model = MS([poly('horsepower', 2, raw=True)])
quad_func = partial(boot_OLS,
                    quad_model,
                    'mpg')
boot_SE(quad_func, Auto, B=1000)


intercept                                  2.067840
poly(horsepower, degree=2, raw=True)[0]    0.033019
poly(horsepower, degree=2, raw=True)[1]    0.000120
dtype: float64

We  compare the results to the standard errors computed using `sm.OLS()`.

In [77]:
M = sm.OLS(Auto['mpg'],
           quad_model.fit_transform(Auto))
summarize(M.fit())['std err']


intercept                                  1.800
poly(horsepower, degree=2, raw=True)[0]    0.031
poly(horsepower, degree=2, raw=True)[1]    0.000
Name: std err, dtype: float64